In [3]:
import os
import sys 
spark_name = os.environ.get('SPARK_HOME',None)
PYSPARK_PYTHON = "D://Unknown softwares//Anaconda//envs//jupyter-ai//python"
# 当存在多个python版本环境时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON
if not spark_name:
    raise ValueError('SPARK_HOME not set')
sys.path.insert(0,os.path.join(spark_name,'python'))
sys.path.insert(0,os.path.join(spark_name,'python/lib/py4j-0.10.9.7-src.zip '))
exec(open(os.path.join(spark_name,'python/pyspark/shell.py')).read())
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.host","localhost").appName("SparkDemo")\
.master("yarn")\
.getOrCreate()

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/

Using Python version 3.11.5 (main, Sep 11 2023 13:26:23)
Spark context Web UI available at http://172.24.2.106:4040
Spark context available as 'sc' (master = local[*], app id = local-1700063546629).
SparkSession available as 'spark'.


In [4]:
df = spark.read.csv('file:///D:/大三上/大数据系统原理与应用/影评情感分析/DMSC.csv', header=True, inferSchema=True, escape="\"", multiLine=True)
df.show()


+---+--------------------+-------------+----------+------+----------------+----------+----+------------------------------------+----+
| ID|       Movie_Name_EN|Movie_Name_CN|Crawl_Date|Number|        Username|      Date|Star|                             Comment|Like|
+---+--------------------+-------------+----------+------+----------------+----------+----+------------------------------------+----+
|  0|Avengers Age of U...|  复仇者联盟2|2017-01-22|     1|            然潘|2015-05-13|   3|          连奥创都知道整容要去韩国。|2404|
|  1|Avengers Age of U...|  复仇者联盟2|2017-01-22|     2|      更深的白色|2015-04-24|   2| 非常失望，剧本完全敷衍了事，主线...|1231|
|  2|Avengers Age of U...|  复仇者联盟2|2017-01-22|     3|    有意识的贱民|2015-04-26|   2|     2015年度最失望作品。以为面面...|1052|
|  3|Avengers Age of U...|  复仇者联盟2|2017-01-22|     4|  不老的李大爷耶|2015-04-23|   4|   《铁人2》中勾引钢铁侠，《妇联1...|1045|
|  4|Avengers Age of U...|  复仇者联盟2|2017-01-22|     5|         ZephyrO|2015-04-22|   2|  虽然从头打到尾，但是真的很无聊啊。| 723|
|  5|Avengers Age of U...|  复仇者联盟2|2017-01-22

In [5]:
data = df.select('Comment', 'star')
data.head()

Row(Comment=' 连奥创都知道整容要去韩国。', star=3)

In [6]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import jieba
word_udf = udf(lambda x: list("/".join(jieba.cut_for_search(x)).split("/")), ArrayType(StringType()))
data = data.withColumn('words', word_udf('Comment'))

In [7]:
from pyspark.ml.feature import HashingTF,IDF
hashingTF = HashingTF(inputCol="words", outputCol="tfFeatures")
tf_df = hashingTF.transform(data)

tf_df.head()

Row(Comment=' 连奥创都知道整容要去韩国。', star=3, words=[' ', '连', '奥创', '都', '知道', '整容', '要', '去', '韩国', '。'], tfFeatures=SparseVector(262144, {42071: 1.0, 59328: 1.0, 61385: 1.0, 74331: 1.0, 146416: 1.0, 167159: 1.0, 186636: 1.0, 208750: 1.0, 211921: 1.0, 239248: 1.0}))

In [8]:
idf = IDF(inputCol="tfFeatures", outputCol="features")
idfModel = idf.fit(tf_df)
idf_df = idfModel.transform(tf_df)

idf_df.head()

Row(Comment=' 连奥创都知道整容要去韩国。', star=3, words=[' ', '连', '奥创', '都', '知道', '整容', '要', '去', '韩国', '。'], tfFeatures=SparseVector(262144, {42071: 1.0, 59328: 1.0, 61385: 1.0, 74331: 1.0, 146416: 1.0, 167159: 1.0, 186636: 1.0, 208750: 1.0, 211921: 1.0, 239248: 1.0}), features=SparseVector(262144, {42071: 2.9817, 59328: 4.9502, 61385: 3.3849, 74331: 1.9807, 146416: 0.0, 167159: 3.5969, 186636: 5.249, 208750: 7.9998, 211921: 0.9291, 239248: 6.6791}))

In [9]:
trainSet, testSet = idf_df.randomSplit([0.9, 0.1])

In [10]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol="features", labelCol="star", smoothing=1.0)
model = nb.fit(trainSet)

In [11]:
# save model
model.save("D:/大三上/大数据系统原理与应用/影评情感分析/Storage/model")

Py4JJavaError: An error occurred while calling o211.save.
: java.io.IOException: Path D:/大三上/大数据系统原理与应用/影评情感分析/Storage/model already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.e

In [12]:
# load model

from pyspark.ml.classification import NaiveBayesModel
NaiveBayesModel.load("D:/大三上/大数据系统原理与应用/影评情感分析/Storage/model")

NaiveBayesModel: uid=NaiveBayes_6dfa58e2e5f1, modelType=multinomial, numClasses=5, numFeatures=262144

In [ ]:
result = model.transform(testSet)
result

In [ ]:
result.select("Comment", "star", "prediction").write.csv(path='', sep="[@")
df = spark.read.csv('/result/part-00000-95c2b7a4-4780-40c6-af29-d81df4998a55-c000.csv',inferShema=True, escape="\"", multiLine=True, sep="[@]")
df.head()

In [ ]:
int2Float_udf = udf(lambda x: float(x))
result_df = df.withColumn("star", int2Float_udf("_c1"))

result_df.head()

In [ ]:
result_df.count()
result_df.filter("star=_c2").count()